<a href="https://colab.research.google.com/github/basakstuff/HAR-using-Smartphones/blob/main/DataMiningv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/DataMining

In [ ]:
# To store data
import pandas as pd

# To do linear algebra
import numpy as np
from numpy import pi

# To create plots
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle

# To create nicer plots
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle

# To get new datatypes and functions
from collections import Counter
from cycler import cycler

# To investigate distributions
from scipy.stats import norm, skew, probplot
from scipy.optimize import curve_fit

# To build models
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import sklearn.neural_network as nn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold,cross_val_score
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier,AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.svm import LinearSVC


# To gbm light
from lightgbm import LGBMClassifier

# To measure time
import time

#Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD,Adam
from keras.models import load_model

In [ ]:
# Load datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Combine boths dataframes
train['Data'] = 'Train'
test['Data'] = 'Test'
both_df = pd.concat([train, test], axis=0).reset_index(drop=True)
both_df['subject'] = '#' + both_df['subject'].astype(str)

# Create label
label = both_df.pop('Activity')

print('Train:\t{}'.format(train.shape))
print('Test:\t{}\n'.format(test.shape))

# Visu

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Group and count main names of columns
pd.DataFrame.from_dict(Counter([col.split('-')[0].split('(')[0] for col in both_df.columns]), orient='index').rename(columns={0:'count'}).sort_values('count', ascending=False)

In [ ]:
# Get null values and dataframe information
print('Null Values In DataFrame: {}\n'.format(both_df.isna().sum().sum()))
both_df.info()

In [ ]:
train.describe()  #we see that the min = -1 and the max = +1. so no need for scaling

In [ ]:
test.describe()  #we see that the min = -1 and the max = +1. so no need for scaling

In [ ]:
# Plotting data
label_counts = label.value_counts()

# Get colors
n = label_counts.shape[0]
colormap = get_cmap('viridis')
colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

# Create plot
data = go.Bar(x = label_counts.index,
              y = label_counts,
              marker = dict(color = colors))

layout = go.Layout(title = 'Smartphone Activity Label Distribution',
                   xaxis = dict(title = 'Activity'),
                   yaxis = dict(title = 'Count'))

plt = go.Figure(data=[data], layout=layout)
plt.show()

In [ ]:
# # Create datasets
# tsne_data = both_df.copy()
# data_data = tsne_data.pop('Data')
# subject_data = tsne_data.pop('subject')

# # Scale data
# scl = StandardScaler()
# tsne_data = scl.fit_transform(tsne_data)

# # Reduce dimensions (speed up)
# pca = PCA(n_components=0.9, random_state=3)
# tsne_data = pca.fit_transform(tsne_data)

# # Transform data
# tsne = TSNE(random_state=3)
# tsne_transformed = tsne.fit_transform(tsne_data)


# # Create subplots
# #fig, axarr = plt.subplots(2, figsize=(15,10))

# axarr = plt.subplots(2, figsize=(15,10))
# fig, axarr = axarr[0], axarr[1:]


# ### Plot Activities
# # Get colors
# n = label.unique().shape[0]
# colormap = get_cmap('viridis')
# colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

# # Plot each activity
# for i, group in enumerate(label_counts.index):
#     # Mask to separate sets
#     mask = (label==group).values
#     axarr[0].scatter(x=tsne_transformed[mask][:,0], y=tsne_transformed[mask][:,1], c=colors[i], alpha=0.5, label=group)
# axarr[0].set_title('t-SNE: Activity Visualisation')
# axarr[0].legend()


# ### Plot Subjects
# # Get colors
# n = subject_data.unique().shape[0]
# colormap = get_cmap('gist_ncar')
# colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1/(n-1))]

# # Plot each participant
# for i, group in enumerate(subject_data.unique()):
#     # Mask to separate sets
#     mask = (subject_data==group).values
#     axarr[1].scatter(x=tsne_transformed[mask][:,0], y=tsne_transformed[mask][:,1], c=colors[i], alpha=0.5, label=group)

# axarr[1].set_title('t-SNE: Participant Visualisation')
# plt.show()

In [ ]:
feature_cols = train.columns[: -1]   #exclude the Activity column
#Calculate the correlation values
correlated_values = train[feature_cols].corr()
#stack the data and convert to a dataframe

correlated_values = (correlated_values.stack().to_frame().reset_index()
                    .rename(columns={'level_0': 'Feature_1', 'level_1': 'Feature_2', 0:'Correlations'}))
correlated_values.head()

In [ ]:
#create an abs_correlation column
correlated_values['abs_correlation'] = correlated_values.Correlations.abs()
correlated_values.head()

In [ ]:
#Picking most correlated features
train_fields = correlated_values.sort_values('Correlations', ascending = False).query('abs_correlation>0.8')
train_fields.sample(5)

In [ ]:
# import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'Dejavu Sans'

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Data provided by each user', fontsize=20)
sns.countplot(x='subject',hue='Activity', data = train)
plt.show()

# Functions


In [ ]:
# get X_train and y_train from csv files
X_train = train.drop(['subject', 'Activity', 'Data'], axis=1)
y_train = train.Activity

In [ ]:
X_train.head()

In [ ]:
# get X_test and y_test from test csv file
X_test = test.drop(['subject', 'Activity', 'Data'], axis=1)
y_test = test.Activity

In [ ]:
print('X_train and y_train : ({},{})'.format(X_train.shape, y_train.shape))
print('X_test  and y_test  : ({},{})'.format(X_test.shape, y_test.shape))

**Labels that are useful in plotting confusion matrix**

In [ ]:
labels=['LAYING', 'SITTING','STANDING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']

### **Function to plot the confusion matrix**

In [ ]:

plt.rcParams["font.family"] = 'DejaVu Sans'

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Purples):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### **Generic function to run any model specified**

In [ ]:
from datetime import datetime
def perform_model(model, X_train, y_train, X_test, y_test, class_labels, cm_normalize=True, average='micro',\
                 print_cm=True, cm_cmap=plt.cm.Oranges):
    
    
    # to store results at various phases
    results = dict()
    
    # time at which model starts training 
    train_start_time = datetime.now()
    print('training the model..')
    model.fit(X_train, y_train)
    print('Done \n \n')
    train_end_time = datetime.now()
    results['training_time'] =  train_end_time - train_start_time
    print('training_time(HH:MM:SS.ms) - {}\n\n'.format(results['training_time']))
    
    
    # predict test data
    print('Predicting test data')
    test_start_time = datetime.now()
    y_pred = model.predict(X_test)
    test_end_time = datetime.now()
    print('Done \n \n')
    results['testing_time'] = test_end_time - test_start_time
    print('testing time(HH:MM:SS:ms) - {}\n\n'.format(results['testing_time']))
    results['predicted'] = y_pred
   

    # calculate overall accuracy of the model
    accuracy = metrics.accuracy_score(y_true=y_test, y_pred=y_pred)
    # store accuracy in results
    results['accuracy'] = accuracy
    print('---------------------')
    print('|      Accuracy      |')
    print('---------------------')
    print('\n    {}\n\n'.format(accuracy))


    # calculate overall F1 score of the model
    f1_score = metrics.f1_score(y_true=y_test, y_pred=y_pred, average='micro')
    # store accuracy in results
    results['f1_score'] = f1_score
    print('---------------------')
    print('|      F1 Score      |')
    print('---------------------')
    print('\n    {}\n\n'.format(f1_score))

    
    # confusion matrix
    cm = metrics.confusion_matrix(y_test, y_pred)
    results['confusion_matrix'] = cm
    if print_cm: 
        print('--------------------')
        print('| Confusion Matrix |')
        print('--------------------')
        print('\n {}'.format(cm))
        
    # plot confusin matrix
    plt.figure(figsize=(8,8))
    plt.grid(b=False)
    plot_confusion_matrix(cm, classes=class_labels, normalize=True, title='Normalized confusion matrix', cmap = cm_cmap)
    plt.show()
    
    # get classification report
    print('-------------------------')
    print('| Classifiction Report |')
    print('-------------------------')
    classification_report = metrics.classification_report(y_test, y_pred)
    # store report in results
    results['classification_report'] = classification_report
    print(classification_report)
    
    # add the trained  model to the results
    results['model'] = model
    
    return results


### **Function to plot Train and Test AUC Curve**

In [ ]:
from sklearn.metrics import roc_curve, auc

def best_model(model, X_train, Y_train, X_test, Y_test):
  classifier = model.best_estimator_
  print(classifier)
  classifier.fit(X_train, Y_train)

  try:
    y_train_pred = classifier.predict_proba(X_train)[:,1]
    y_test_pred = classifier.predict_proba(X_test)[:,1]
  except:
    y_train_pred = classifier.decision_function(X_train)
    y_test_pred = classifier.decision_function(X_test)

  train_fpr, train_tpr, tr_thresholds = roc_curve(Y_train, y_train_pred)
  test_fpr, test_tpr, te_thresholds = roc_curve(Y_test, y_test_pred)

  ### --- This is test AUC for TFIDF
  auc_bow = auc(test_fpr, test_tpr)

  plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
  plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
  plt.legend()
  plt.xlabel("FPR")
  plt.ylabel("TPR")
  plt.title("ERROR PLOTS")
  plt.grid()
  plt.show()

### **Method to print the gridsearch Attributes**

In [ ]:
def print_grid_search_attributes(model):
    # Estimator that gave highest score among all the estimators formed in GridSearch
    print('--------------------------')
    print('|      Best Estimator     |')
    print('--------------------------')
    print('\n\t{}\n'.format(model.best_estimator_))


    # parameters that gave best results while performing grid search
    print('--------------------------')
    print('|     Best parameters     |')
    print('--------------------------')
    print('\tParameters of best estimator : \n\n\t{}\n'.format(model.best_params_))


    #  number of cross validation splits
    print('---------------------------------')
    print('|   No of CrossValidation sets   |')
    print('--------------------------------')
    print('\n\tTotal number of cross validation sets: {}\n'.format(model.n_splits_))


    # Average cross validated score of the best estimator, from the Grid Search 
    print('--------------------------')
    print('|        Best Score       |')
    print('--------------------------')
    print('\n\tAverage Cross Validate scores of best estimator : \n\n\t{}\n'.format(model.best_score_))

# Models


## 1. Logistic Regression with Grid Search


In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('X_train_scaled and y_train : ({},{})'.format(X_train_scaled.shape, y_train.shape))
print('X_test_scaled and y_test  : ({},{})'.format(X_test_scaled.shape, y_test.shape))

In [ ]:

# start Grid search
parameters_LR = {'C':[0.01, 0.1, 1, 10, 20, 30], 'penalty':['l2','l1']}
log_reg = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)
log_reg_grid = GridSearchCV(log_reg, param_grid=parameters_LR, cv=10, verbose=1, return_train_score=True, n_jobs=-1)
log_reg_grid_results =  perform_model(log_reg_grid, X_train, y_train, X_test, y_test, class_labels=labels)

In [ ]:

plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(log_reg_grid_results['confusion_matrix'], classes=labels, cmap=plt.cm.Purples, )
plt.show()

In [ ]:
max_scores = pd.DataFrame(log_reg_grid.cv_results_).groupby(['param_penalty','param_C']).max().unstack()[['mean_test_score', 'mean_train_score']]

fig, ax = plt.subplots(1,2, figsize=(20,6))

sns.heatmap(max_scores.mean_train_score, annot = True, fmt='.4g', ax=ax[0])
sns.heatmap(max_scores.mean_test_score, annot = True, fmt='.4g', ax=ax[1])

ax[0].set_title('Train Set')
ax[1].set_title('CV Set')

plt.show()

In [ ]:
# observe the attributes of the model 
print_grid_search_attributes(log_reg_grid_results['model'])

In [ ]:
print(log_reg_grid.best_estimator_)

### ?????? comment


In [ ]:
ModelBest = log_reg_grid.best_estimator_
classifier = ModelBest
classifier.fit(X_train_scaled, y_train)

In [ ]:
y_train_pred = classifier.predict_proba(X_train_scaled)
print(y_train_pred.shape)

In [ ]:

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

y_train_dummies = pd.get_dummies(y_train, drop_first=False).values

for i in range(6):
    fpr[i], tpr[i],_ = roc_curve(y_train_dummies[:, i], y_train_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# roc for each class
fig, ax = plt.subplots(figsize=(17, 6))
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
for i in range(6):
    ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
ax.legend(loc="best")
ax.grid(alpha=.4)
sns.despine()
plt.show()

## 2. Linear SVC with GridSearch


**Rapor için not:**



  Whether to enable probability estimates. This must be enabled prior to calling fit, will slow down that method as it internally uses 5-fold cross-validation, and predict_proba may be inconsistent with predict. Read more in the User Guide.

In [ ]:
parameters_LSVC = {'C':[0.125, 0.5, 1, 2, 8, 16]}
lr_svc = LinearSVC(tol=0.00005)
lr_svc_grid = GridSearchCV(lr_svc, param_grid=parameters_LSVC, cv=10, n_jobs=-1, verbose=1, return_train_score=True)
lr_svc_grid_results = perform_model(lr_svc_grid, X_train, y_train, X_test, y_test, class_labels=labels)

In [ ]:
plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(lr_svc_grid_results['confusion_matrix'], classes=labels, cmap=plt.cm.Purples, )
plt.show()

In [ ]:
print_grid_search_attributes(lr_svc_grid_results['model'])

## 3. Kernel SVM with GridSearch

In [ ]:
from sklearn.svm import SVC
parameters_ksvm = {'C':[2,8,16],\
              'gamma': [ 0.0078125, 0.125, 2]}
rbf_svm = SVC(kernel='rbf')
rbf_svm_grid = GridSearchCV(rbf_svm, param_grid=parameters_ksvm, cv=10, n_jobs=-1)
rbf_svm_grid_results = perform_model(rbf_svm_grid, X_train, y_train, X_test, y_test, class_labels=labels)

In [ ]:
print_grid_search_attributes(rbf_svm_grid_results['model'])

## 4. Decision Trees with GridSearchCV

In [ ]:
parameters_DT = {'max_depth':np.arange(3,10,2)}
dt = DecisionTreeClassifier()
dt_grid = GridSearchCV(dt,param_grid=parameters_DT, cv=10, n_jobs=-1, cv=10)
dt_grid_results = perform_model(dt_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(dt_grid_results['model'])

In [ ]:

print_grid_search_attributes(dt_grid_results['model'])

## 5. Random Forest Classifier with GridSearch

In [ ]:

from sklearn.ensemble import RandomForestClassifier
params_RF = {'n_estimators': np.arange(10,201,20), 'max_depth':np.arange(3,15,2)}
rfc = RandomForestClassifier()
rfc_grid = GridSearchCV(rfc, cv=10, param_grid=params_RF, n_jobs=-1)
rfc_grid_results = perform_model(rfc_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(rfc_grid_results['model'])

## 6. Gradient Boosted Decision Trees With GridSearch

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
param_grid_GBDT = {'max_depth': np.arange(5,8,1), \
             'n_estimators':np.arange(130,170,10)}
gbdt = GradientBoostingClassifier()
gbdt_grid = GridSearchCV(gbdt, cv=10, param_grid=param_grid_GBDT, n_jobs=-1)
gbdt_grid_results = perform_model(gbdt_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(gbdt_grid_results['model'])

## 7. Light GBM with GridSearch

In [ ]:
from lightgbm import LGBMClassifier
parameters_lgbm = {'max_depth': [1, 5, 10, 50], 'n_estimators':np.arange(10,121,20)}

lgbm = LGBMClassifier()
lgbm_grid = GridSearchCV(lgbm,param_grid=parameters_lgbm, cv=10, verbose=1, n_jobs=-1, return_train_score=True)
lgbm_grid_results = perform_model(lgbm_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(lgbm_grid_results['model'])

## 8. XGBoost with GridSearch

In [ ]:

from xgboost import XGBClassifier
parameters_xgb = {'max_depth': [1, 5, 10, 50], 'n_estimators':np.arange(10,121,20)}

xgb = XGBClassifier()
xgb_grid = GridSearchCV(xgb,param_grid=parameters_xgb, cv=10, verbose=1, n_jobs=-1, return_train_score=True)
xgb_grid_results = perform_model(xgb_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(xgb_grid_results['model'])

## 9. KNN with Grid Search

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# start Grid search
parameters_KNN = {'n_neighbors':[3, 15, 25, 51, 101, 151, 201, 301, 401]}
KNN_Class = KNeighborsClassifier()
KNN_Class_grid = GridSearchCV(KNN_Class, param_grid=parameters_KNN, cv=10, verbose=1, n_jobs=-1, return_train_score=True)
KNN_Class_grid_results =  perform_model(KNN_Class_grid, X_train, y_train, X_test, y_test, class_labels=labels)
print_grid_search_attributes(KNN_Class_grid_results['model'])

# Comparing Models

In [ ]:

print('\n                     Accuracy     Error     F1 Score')
print('                     ----------   --------   ---------')
print('Logistic Regression : {:.04}%       {:.04}%    {:.04}%'.format(log_reg_grid_results['accuracy'] * 100,\
                                                  100-(log_reg_grid_results['accuracy'] * 100),\
                                                  (log_reg_grid_results['f1_score'] * 100)))

print('Linear SVC          : {:.04}%       {:.04}%    {:.04}%'.format(lr_svc_grid_results['accuracy'] * 100,\
                                                        100-(lr_svc_grid_results['accuracy'] * 100),\
                                                  (lr_svc_grid_results['f1_score'] * 100)))

print('rbf SVM classifier  : {:.04}%      {:.04}%    {:.04}%'.format(rbf_svm_grid_results['accuracy'] * 100,\
                                                          100-(rbf_svm_grid_results['accuracy'] * 100),\
                                                  (rbf_svm_grid_results['f1_score'] * 100)))

print('DecisionTree        : {:.04}%      {:.04}%    {:.04}%'.format(dt_grid_results['accuracy'] * 100,\
                                                        100-(dt_grid_results['accuracy'] * 100),\
                                                  (dt_grid_results['f1_score'] * 100)))

print('Random Forest       : {:.04}%      {:.04}%    {:.04}%'.format(rfc_grid_results['accuracy'] * 100,\
                                                           100-(rfc_grid_results['accuracy'] * 100),\
                                                  (rfc_grid_results['f1_score'] * 100)))

print('GradientBoosting DT : {:.04}%      {:.04}%    {:.04}%'.format(gbdt_grid_results['accuracy'] * 100,\
                                                        100-(gbdt_grid_results['accuracy'] * 100),\
                                                  (gbdt_grid_results['f1_score'] * 100)))

print('Light GBM           : {:.04}%      {:.04}%    {:.04}%'.format(lgbm_grid_results['accuracy'] * 100,\
                                                        100-(lgbm_grid_results['accuracy'] * 100),\
                                                  (lgbm_grid_results['f1_score'] * 100)))

print('XGBoost             : {:.04}%     {:.04}%    {:.04}%'.format(xgb_grid_results['accuracy'] * 100,\
                                                        100-(xgb_grid_results['accuracy'] * 100),\
                                                  (xgb_grid_results['f1_score'] * 100)))

print('KNN                 : {:.04}%      {:.04}%    {:.04}%'.format(KNN_Class_grid_results['accuracy'] * 100,\
                                                        100-(KNN_Class_grid_results['accuracy'] * 100,\
                                                  (KNN_Class_grid_results['f1_score'] * 100)))



